### 1. Load libraries

In [1]:
from scoremtb.score import compute_score as cs
from scoremtb.score import get_syndata as gsyn
import pandas as pd

### 2. Load dataset

In [2]:
PROJECT_ID = 'syn29357579'

In [3]:
#Load data from S3
# syn = gsyn.get_synclient()
# dfid = gsyn.parquet_2_df(syn, PROJECT_ID, 'dataset_archivemetadata_v1/')
# stepdata = gsyn.parquet_2_df(syn, PROJECT_ID, 'dataset_sharedschema_v1_steps/')
# task_data = gsyn.parquet_2_df(syn, PROJECT_ID, 'dataset_sharedschema_v1/')
# task_status = gsyn.parquet_2_df(syn, PROJECT_ID, 'dataset_sharedschema_v1_taskStatus/')

######## OR ###########
#Load data from downloaded csv's
dfid = pd.read_csv('data/dfid.csv', low_memory=False)
stepdata = pd.read_csv('data/stepdata.csv', low_memory=False)
task_data = pd.read_csv('data/task_data.csv', low_memory=False)
task_status = pd.read_csv('data/task_status.csv', low_memory=False)

In [4]:
#Merge task status
task_status = task_status.drop(columns=['id', 'index'])
dfid = dfid.merge(task_status, on = ['recordid', 'assessmentid', 'year', 'month', 'day'],
                    how='left').reset_index(drop=True)

### 3. Filter Scores

##### 3.1 Study membership details

In [5]:
def study_membership():
    syn = gsyn.get_synclient('USER_NAME', 'PASSWORD') 
    
    table_syn = syn.tableQuery('SELECT * FROM syn29300177')
    study_details = table_syn.asDataFrame()
    study_details = study_details.sort_values(by=['healthCode', 'participantVersion']).reset_index(drop=True)
    
    study_details = study_details[['healthCode','studyMemberships']]
    study_details = study_details.rename(columns={"healthCode": "healthcode"})
    
    study_details = study_details.drop_duplicates(keep='last')
    return study_details.reset_index(drop=True)

In [18]:
study_reference = study_membership()

#### 3.2 Spelling

In [7]:
spelling_score = cs.get_score(task_data, stepdata, dfid, 'spelling', study_reference)

#### 3.3 Vocab

In [8]:
vocab_score = cs.get_score(task_data, stepdata, dfid, 'vocabulary', study_reference)

#### 3.4 PSM

In [19]:
PSM_score = cs.get_score(task_data, stepdata, dfid, 'psm', study_reference)

#### 3.5 MFS

In [10]:
score_mfs = cs.get_score(task_data, stepdata, dfid, 'memory-for-sequences', study_reference)

#### 3.6 DCCS

In [11]:
score_dccs = cs.get_score(task_data, stepdata, dfid, 'dccs', study_reference)

#### 3.7 Fname

In [12]:
score_fname = cs.get_score(task_data, stepdata, dfid, 'fnameb', study_reference)

#### 3.8 Number Match

In [13]:
score_nm = cs.get_score(task_data, stepdata, dfid, 'number-match', study_reference)

#### 3.9 Flanker

In [14]:
score_flanker = cs.get_score(task_data, stepdata, dfid, 'flanker', study_reference)

### 4. Combine Score

In [15]:
stack_merged = cs.combine_scores([score_mfs, score_dccs, score_fname, score_nm, score_flanker, PSM_score, 
                                   spelling_score, vocab_score], dfid)

In [17]:
# stack_merged.to_csv('MTB_UCSF_scores.csv', index=False)